# 事前準備

ライブラリのインストール

In [ ]:
!pip install numexpr

Googleドライブのマウント

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 港湾統計マスタの前処理

**注意点**

*   港区の概念は無視（港区が存在する港、存在しない港がある）
*   港湾名に統一（港名という表現は変換する）



## 公共施設・占用施設

公共施設コード.csv、専用施設コード.csvを事前に作成しておく

施設コードが重複している場合は、代表施設だけの1行に加工

In [15]:
import pandas as pd

def quay(path):
  
  # csvのインポート
  df = pd.read_csv(path, header=0, encoding="shift-jis",dtype=str)
  
  # 欠損行の削除
  df = df.dropna(how='all')
  
  # 公共施設・専用施設をマージ
  df_public = pd.read_csv('/content/drive/MyDrive/港湾統計（兵庫県）/公共施設コード.csv', header=0, encoding="shift-jis",dtype=str)
  df_private = pd.read_csv('/content/drive/MyDrive/港湾統計（兵庫県）/専用施設コード.csv', header=0, encoding="shift-jis",dtype=str)  
  df = pd.merge(df, pd.concat([df_public, df_private]), left_on='施設', right_on='施設コード', how='left')

  # 必要な列のみ抽出
  columns = ['施設','港湾名','施設名','岸壁種別']
  df = df[columns]

  return df

In [16]:
path = '/content/drive/MyDrive/港湾統計（兵庫県）/港湾統計マスタ_2021.csv'

df_quay = quay(path)
df_quay

,施設,港湾名,施設名,岸壁種別
0,2350,姫路港,姫路ＬＮＧ施設,専用岸壁
1,1241,姫路港,今在家（入船）岸壁(-5.5m),公共岸壁
2,2308,姫路港,合同製鐵（−９．０ｍ）施設,専用岸壁
3,2352,姫路港,ＪＦＥ条鋼施設,専用岸壁
4,1212,姫路港,中島公共岸壁(-5.5m),公共岸壁
...,...,...,...,...
44822,2488,赤穂港,住友大阪セメント施設,専用岸壁
44823,2488,赤穂港,住友大阪セメント施設,専用岸壁
44824,2488,赤穂港,住友大阪セメント施設,専用岸壁
44825,2488,赤穂港,住友大阪セメント施設,専用岸壁


## 取扱貨物量

In [33]:
def cargo(path):

  # csvのインポート
  df = pd.read_csv(path, header=0, encoding="shift-jis",dtype=str)
  
  # 欠損行の削除
  df = df.dropna(how='all')

  # 欠損値の置換
  df = df.fillna('-')

  # 整数値に変換
  df = df.astype({'入荷ﾄﾝ数': 'int','出荷ﾄﾝ数': 'int'})

  # 取扱貨物区分を返却する関数
  def func1(row):
    if row['入荷内外'] != '-':
      return '輸入' if row['入荷内外'] == '2' else '移入'
    if row['出荷内外'] != '-':
      return '輸出' if row['出荷内外'] == '2' else '移出'

  # 取扱貨物量を返却する関数
  def func2(row):
    if row['入荷ﾄﾝ数'] != 0:
      return row['入荷ﾄﾝ数']
    if row['出荷ﾄﾝ数'] != 0:
      return row['出荷ﾄﾝ数']

  # 相手港コードを返却する関数
  def func3(row):
    if row['入荷仕出港'] != '-':
      return row['入荷仕出港']
    if row['出荷仕向港'] != '-':
      return row['出荷仕向港']
  
  # 輸出入・移出入に区分
  df['取扱貨物区分'] = df.apply(func1, 1)
  df['取扱貨物量'] = df.apply(func2, 1)
  df['相手港コード'] = df.apply(func3, 1)

  
  # 品名をマージ
  df['品名'] = df.apply(lambda x: x['出荷品名'] if x['出荷ﾄﾝ数'] != 0 else x['入荷品名'], 1)
  df1 = pd.read_csv('/content/drive/MyDrive/港湾統計（兵庫県）/品名コード.csv', header=0, encoding="shift-jis",dtype=str)
  df = pd.merge(df, df1, on='品名', how='left')

  # 必要な列のみ抽出
  columns = ['施設','中分類','大分類','取扱貨物区分','取扱貨物量','相手港コード']
  df = df[columns]

  return df

In [27]:
path = '/content/drive/MyDrive/港湾統計（兵庫県）/港湾統計マスタ_2021.csv'

df_cargo = cargo(path)
df_cargo

,施設,品名,中分類,大分類,取扱貨物区分,取扱貨物量,相手港コード
0,2350,322,ＬＮＧ（液化天然ガス）,(5)化学工業品,輸入,187122.0,2101015
1,1241,161,砂利・砂,(3)鉱産品,移入,1600.0,28023
2,2308,481,金属くず,(8)特殊品,移入,713.0,33003
3,2352,222,鋼材,(4)金属機械工業品,移出,469.0,40002
4,1212,281,セメント,(5)化学工業品,移入,1502.0,28014
...,...,...,...,...,...,...,...
44822,2488,131,石炭,(3)鉱産品,輸入,9940.0,5105998
44823,2488,191,石灰石,(3)鉱産品,移入,12860.0,35032
44824,2488,281,セメント,(5)化学工業品,移出,3514.0,30998
44825,2488,281,セメント,(5)化学工業品,移出,2802.0,12998


## 係留船舶

In [9]:
import pandas as pd

def ship(path):

  # csvのインポート
  df = pd.read_csv(path, header=0, encoding="shift-jis",dtype=str)
  
  # 欠損行の削除
  df = df.dropna(how='all')

  # 隻数のセット
  df = df.astype({'船舶隻数': 'int','隻数A': 'int','隻数B': 'int'})
  df['隻数'] = df[['船舶隻数','隻数A','隻数B']].sum(axis=1)

  # 総トン数のセット
  df = df.astype({'船舶総ﾄﾝ数': 'int','総ﾄﾝ数A': 'int','総ﾄﾝ数B': 'int'})
  df['総トン数'] = df[['船舶総ﾄﾝ数','総ﾄﾝ数A','総ﾄﾝ数B']].sum(axis=1) 

  # 用途のtをTに変換
  df = df.replace('t71', 'T71')
  df = df.replace('t85', 'T85')
  
  # 船種をマージ
  df1 = pd.read_csv('/content/drive/MyDrive/港湾統計（兵庫県）/用途コード.csv', header=0, encoding="shift-jis",dtype=str)
  df = pd.merge(df, df1, on='用途', how='left')

  # 必要な列のみ抽出
  columns = ['施設','隻数','総トン数','船種','係留時間']
  df = df[columns]


  return df

In [10]:
path = '/content/drive/MyDrive/港湾統計（兵庫県）/港湾統計マスタ_2021.csv'

df_ship = ship(path)
df_ship

,施設,隻数,総トン数,船種,係留時間
0,2350,1,113502,ＬＮＧ船,23
1,1241,1,480,砂利・砂・石材船,3
2,2308,1,19,曳船・押船,13
3,2352,1,376,一般貨物船,28
4,1212,1,749,セメント船,4
...,...,...,...,...,...
44822,2488,1,9995,石炭船,24
44823,2488,1,8858,その他専用船,26
44824,2488,1,3118,セメント船,5
44825,2488,0,0,NaN,0


## 整形後の港湾統計マスタ

In [35]:
import pandas as pd

def master(path):

  # 施設情報
  df_quay = quay(path)

  # 取扱貨物量
  df_cargo = cargo(path).drop('施設', axis=1)

  df = pd.merge(df_quay, df_cargo, left_index=True, right_index=True, how='left')

  # 係留船舶
  df_ship = ship(path).drop('施設', axis=1)

  df = pd.merge(df, df_ship, left_index=True, right_index=True, how='left')
    
  return df

In [36]:
path = '/content/drive/MyDrive/港湾統計（兵庫県）/港湾統計マスタ_2021.csv'

df_master = master(path)
df_master

,施設,港湾名,施設名,岸壁種別,中分類,大分類,取扱貨物区分,取扱貨物量,相手港コード,隻数,総トン数,船種,係留時間
0,2350,姫路港,姫路ＬＮＧ施設,専用岸壁,ＬＮＧ（液化天然ガス）,(5)化学工業品,輸入,187122.0,2101015,1,113502,ＬＮＧ船,23
1,1241,姫路港,今在家（入船）岸壁(-5.5m),公共岸壁,砂利・砂,(3)鉱産品,移入,1600.0,28023,1,480,砂利・砂・石材船,3
2,2308,姫路港,合同製鐵（−９．０ｍ）施設,専用岸壁,金属くず,(8)特殊品,移入,713.0,33003,1,19,曳船・押船,13
3,2352,姫路港,ＪＦＥ条鋼施設,専用岸壁,鋼材,(4)金属機械工業品,移出,469.0,40002,1,376,一般貨物船,28
4,1212,姫路港,中島公共岸壁(-5.5m),公共岸壁,セメント,(5)化学工業品,移入,1502.0,28014,1,749,セメント船,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44822,2488,赤穂港,住友大阪セメント施設,専用岸壁,石炭,(3)鉱産品,輸入,9940.0,5105998,1,9995,石炭船,24
44823,2488,赤穂港,住友大阪セメント施設,専用岸壁,石灰石,(3)鉱産品,移入,12860.0,35032,1,8858,その他専用船,26
44824,2488,赤穂港,住友大阪セメント施設,専用岸壁,セメント,(5)化学工業品,移出,3514.0,30998,1,3118,セメント船,5
44825,2488,赤穂港,住友大阪セメント施設,専用岸壁,セメント,(5)化学工業品,移出,2802.0,12998,0,0,NaN,0


# 県全体

## 港湾名・中分類✕輸出入

In [41]:
import pandas as pd
import numpy as np

path = '/content/drive/MyDrive/港湾統計（兵庫県）/港湾統計マスタ_2021.csv'

df_master = master(path)
df_master

df = pd.pivot_table(df_master, index=['港湾名', '中分類'], columns='取扱貨物区分', values='総トン数', margins=True, aggfunc=np.sum).reset_index()
df

取扱貨物区分,港湾名,中分類,移入,移出,輸入,輸出,All
0,姫路港,その他石油製品,92358.0,499.0,23808.0,NaN,116665
1,姫路港,その他繊維工業品,NaN,499.0,NaN,NaN,499
2,姫路港,その他輸送機械,NaN,13211.0,NaN,5259.0,18470
3,姫路港,その他農産品,2522.0,NaN,NaN,NaN,2522
4,姫路港,とうもろこし,4257.0,NaN,NaN,NaN,4257
...,...,...,...,...,...,...,...
126,赤穂港,非金属鉱物,92951.0,NaN,68337.0,NaN,161288
127,赤穂港,再利用資材,56299.0,NaN,NaN,NaN,56299
128,赤穂港,石灰石,2248523.0,NaN,NaN,NaN,2248523
129,赤穂港,その他食料工業品,NaN,21234.0,NaN,NaN,21234


# 姫路港・東播磨港

**前準備**
*   調査港コード.csvの作成

In [3]:
def get_harima(path,columns):

  # csvのインポート
  df = pd.read_csv(path, header=0, encoding="shift-jis",dtype=str)

  # 空白行を削除
  df = df.dropna(subset=['調査年'])

  # 抽出する列のリスト指定
  df = df[columns]
  
  # 調査港をマージ
  df2 = pd.read_csv('/content/drive/MyDrive/港湾統計（兵庫県）/調査港コード.csv', header=0, encoding="shift-jis",dtype=str)
  df = pd.merge(df, df2, on='調査港', how='right')

  return df

In [4]:
path = '/content/drive/MyDrive/港湾統計（兵庫県）/港湾統計マスタ_2021.csv'
columns = ['調査港','施設','公専','用途','船舶隻数','船舶総ﾄﾝ数','係留時間','出荷内外','出荷品名','出荷ﾄﾝ数','出荷仕向港','入荷内外','入荷品名','入荷ﾄﾝ数','入荷仕出港']

df_master =  get_harima(path,columns)
df_master

NameError: ignored

# 播磨地域CNP

東播磨港のうち明石市・播磨町は対象外

**前準備**
*   公共岸壁コード.csvの作成（播磨CNP対象岸壁）
*   専用岸壁コード.csvの作成（播磨CNP対象岸壁）




In [ ]:
def get_cnp(path,columns):

  # csvのインポート
  df = pd.read_csv(path, header=0, encoding="shift-jis",dtype=str)

  # 空白行を削除
  df = df.dropna(subset=['調査年'])

  # 抽出する列のリスト指定
  df = df[columns]
  
  # 調査港をマージ
  df2 = pd.read_csv('/content/drive/MyDrive/港湾統計（兵庫県）/調査港コード.csv', header=0, encoding="shift-jis",dtype=str)
  df = pd.merge(df, df2, on='調査港', how='right')

  # 公共岸壁・専用岸壁をマージ
  df_public = pd.read_csv('/content/drive/MyDrive/港湾統計（兵庫県）/公共岸壁コード.csv', header=0, encoding="shift-jis",dtype=str)
  df_private = pd.read_csv('/content/drive/MyDrive/港湾統計（兵庫県）/専用岸壁コード.csv', header=0, encoding="shift-jis",dtype=str)  
  df2 = pd.concat([df_public, df_private])
  df = pd.merge(df, df2, on=['施設','港名'], how='left')
  
  return df

In [ ]:
path = '/content/drive/MyDrive/港湾統計（兵庫県）/港湾統計マスタ_2021.csv'
columns = ['調査港','施設','公専','出荷内外','出荷品名','出荷ﾄﾝ数','出荷仕向港','入荷内外','入荷品名','入荷ﾄﾝ数','入荷仕出港','用途','船舶内外','船舶隻数','隻数A','隻数B','船舶総ﾄﾝ数','総ﾄﾝ数A','総ﾄﾝ数B','係留時間']

df_cnp =  get_cnp(path,columns)
df_cnp
